# DEVOPS SERVER

Comandos que serão utilizados no servidor de DEVOPS.

## Configuração Inicial

In [2]:
import os
FILE_DIR = os.path.abspath('../../files')

## Tabelas

In [3]:
import os
from datetime import datetime
from pony import orm

db = orm.Database()

class Tarefa(db.Entity):
    id = orm.PrimaryKey(int, auto=True)
    titulo = orm.Required(str, index=True)
    detalhe = orm.Optional(str, index=True, nullable=True)
    situacao = orm.Required('Situacao', index=True)
    datahora_criacao = orm.Required(datetime, index=True)
    datahora_inicio = orm.Optional(datetime, index=True, nullable=True)
    datahora_fim = orm.Optional(datetime, index=True, nullable=True)
    comandos = orm.Set('Comando')

class Situacao(db.Entity):
    id = orm.PrimaryKey(int)
    descricao = orm.Required(str)
    
class Comando(db.Entity):
    id = orm.PrimaryKey(int, auto=True)
    tarefa = orm.Required(Tarefa, index=True)
    orm.composite_key(id, tarefa)
    comando = orm.Required(str)
    resultados = orm.Set('Resultado')
    datahora_inicio = orm.Optional(datetime, index=True, nullable=True)
    datahora_fim = orm.Optional(datetime, index=True, nullable=True)
    
class Resultado(db.Entity):
    id = orm.PrimaryKey(int, auto=True)
    comando = orm.Required(Comando, index=True)
    orm.composite_key(id, comando)
    resultado_sucesso = orm.Optional(str, nullable=True)
    resultado_erro = orm.Optional(str, nullable=True)

db.bind(provider='sqlite', filename=os.path.join(FILE_DIR, f'devops_tarefas.sqlite'), create_db=True)
db.generate_mapping(create_tables=True)

In [6]:
with orm.db_session:
    Situacao(id=1, descricao='criado')
    Situacao(id=2, descricao='iniciado')
    Situacao(id=3, descricao='finalizado')
    Situacao(id=4, descricao='cancelado')
    orm.commit()


## Console stream

In [7]:
import os
import platform
import re
import subprocess
from datetime import datetime
from pony import orm

if platform.system() == 'Windows':
    os.environ['COMSPEC'] = 'powershell'
    encoding = 'utf-8'
else:
    encoding = 'utf-8'
encoding_error = 'replace' # 'ignore'

REGEX_PULAR_LINHA_1 = re.compile(r'[\r\n]')
REGEX_PULAR_LINHA_2 = re.compile(r'[\r]')

@orm.db_session
def shell_executar(id_: int) -> True:
    tarefa_comando = Comando[id_]
    tarefa_comando.datahora_inicio = datetime.now()
    orm.commit()
    
    situacao_execucao = True
    cmd = tarefa_comando.comando.split(' ')
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    for line in proc.stdout.readlines():
        Resultado(
                comando=Comando[id_],
                resultado_sucesso=REGEX_PULAR_LINHA_1.sub('', line.decode(encoding, encoding_error))
        )
    
    error = REGEX_PULAR_LINHA_2.sub('', proc.stderr.read().decode(encoding, encoding_error))
    if error:
        Resultado(
                comando=Comando[id_],
                resultado_erro=error
        )
        situacao_execucao = False
    
    tarefa_comando = Comando[id_]
    tarefa_comando.datahora_fim = datetime.now()
    
    return situacao_execucao


## Fila e Thread

In [8]:
from datetime import datetime
from queue import Queue
from time import sleep
from pony import orm

@orm.db_session
def processar_fila(fila: Queue) -> None:
    tempo_espera = 1
    tempo_maximo = 5
    tempo_acrescimo = 0.5
    
    while True:
        
        if fila.empty():
            sleep(tempo_espera)
            tempo_espera = tempo_espera + tempo_acrescimo if tempo_espera < tempo_maximo else tempo_espera
            continue
        
        else:
            tempo_espera = 1
        
        id_ = fila.get()
        tarefa = Tarefa[id_]
        tarefa.situacao = Situacao[2]
        tarefa.datahora_inicio = datetime.now()
        orm.commit()

        ultimo_retorno = True
        lista_comandos = Comando.select(lambda c: c.tarefa == Tarefa[id_]).order_by(Comando.id)
        for comando in lista_comandos:
            ultimo_retorno = shell_executar(comando.id)
        
        tarefa = Tarefa[id_]
        tarefa.situacao = Situacao[3] if ultimo_retorno is True else Situacao[4]
        tarefa.datahora_fim = datetime.now()
        orm.commit()


In [9]:
from queue import Queue
from threading import Thread

fila_tarefas = Queue()
fila_tarefas.queue.clear()

thread1 = Thread(target=processar_fila, args=(fila_tarefas,), daemon=True)
thread1.start()


## Inclusão de tarefas com comandos

In [13]:
from datetime import datetime
from typing import Any, List
from pony import orm

@orm.db_session
def execucao_comandos(titulo: str, lista_comando: List[str], **kwargs: Any) -> int:
    tarefa = Tarefa(
        titulo=titulo,
        detalhe=kwargs.get('detalhe'),
        situacao=Situacao[1],
        datahora_criacao=datetime.now()
    )
    for comando in lista_comando:
        Comando(
            tarefa=tarefa,
            comando=comando
        )
    orm.commit()
    id_ = tarefa.id
    fila_tarefas.put(id_)
    return id_


### Execução com sucesso

In [12]:
import time
from pony import orm

lista_comandos = [
    r'cd C:\Users\Ape11\workspace; ls'
]

id_ = execucao_comandos('listando diretorio', lista_comandos)
with orm.db_session:
    time.sleep(5)
    lc = Comando.select(lambda t: t.tarefa==Tarefa[id_]).order_by(Comando.id)
    for c in lc:
        lr = Resultado.select(lambda r: r.comando==c).order_by(Resultado.id)
        for r in lr:
            mensagem = r.resultado_sucesso or r.resultado_erro or ''
            print(mensagem)




Diret�rio: C:\Users\Ape11\workspace


Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----        09/09/2020     11:54                ape11-notebook
d-----        18/09/2020     17:56                aut-api-bff
d-----        04/09/2020     13:43                bof-web-corretor
d-----        09/09/2020     11:06                cdc-postgres
d-----        10/09/2020     11:28                cnt-api-bff
d-----        18/09/2020     14:02                cnt-colocarLogoFoto
d-----        29/07/2020     09:44                cnt-job-publicar-imoveis-portais
d-----        10/08/2020     11:53                cnt-job-solicitar_prospeccao
d-----        29/05/2020     15:29                cnt-melhorarFoto
d-----        05/07/2020     11:35                cnt-solicitar-documentos
d-----        09/09/2020     15:38                cnt-verifaFotos
d-----        18/09/2020     13:56                crm-api-bff
d-----        28/05/2020     

### Execução com erro

In [14]:
import time
from pony import orm

lista_comandos = [
    r'python C:/Users/Ape11/workspace/lab/ok.py',
    r'python C:/Users/Ape11/workspace/lab/error.py'
]

id_ = execucao_comandos('listando diretorio', lista_comandos)
with orm.db_session:
    time.sleep(5)
    lc = Comando.select(lambda t: t.tarefa==Tarefa[id_]).order_by(Comando.id)
    for c in lc:
        lr = Resultado.select(lambda r: r.comando==c).order_by(Resultado.id)
        for r in lr:
            mensagem = r.resultado_sucesso or r.resultado_erro or ''
            print(mensagem)


>> Finalizado com sucesso!
>> num: 31
Traceback (most recent call last):
  File "C:/Users/Ape11/workspace/lab/error.py", line 5, in <module>
    raise Exception('erro')
Exception: erro
